In [1]:
from pathlib import Path
import sys 
import cv2
import pdb
import numpy as np
import os
import copy
import matplotlib.pyplot as plt
from typing import List
import imutils

In [2]:
def show_image(image_,
               window_name='image',
               timeout=0,
               framework='opencv',
               resize=True):
    """
    Show image.
    :param image_
    :param window_name
    :param timeout
    """
    if framework == "opencv":
        if resize:
            # rescale the image to be smaller such that in can fit in the display resolution 
            scale_percent = 20 # percent of original size
            width = int(image_.shape[1] * scale_percent / 100)
            height = int(image_.shape[0] * scale_percent / 100)
            dim = (width, height)
            image_ = cv2.resize(image_, dim, interpolation = cv2.INTER_AREA)
        cv2.imshow(window_name, np.uint8(image_))
        cv2.waitKey(timeout)
        cv2.destroyAllWindows()
    else:
        RGB_image_ = cv2.cvtColor(image_, cv2.COLOR_BGR2RGB)
        plt.axis("off")
        plt.imshow(np.uint8(RGB_image_))
        plt.show()

In [13]:
def get_keypoints_and_features(image, show_details=False, max_features = 1000) -> tuple:
    """
    :return the keypoints: [cv2.Keypoint] and the features: np.ndarray for each keypoint.
    """

    def show_keypoints(image_, keypoints_):
        """
        Show the keypoints found in the image.
        """
        image_output = image_.copy()
        image_output = cv2.drawKeypoints(image_, keypoints_, image_output, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        # show_image(image_output, 'keypoints',framework="matplotlib")
        show_image(image_output, 'Matched keypoints',framework="opencv2")

    # convert image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 

    # create the SIFT object. (https://docs.opencv2.org/master/da/df5/tutorial_py_sift_intro.html)
    orb = cv2.ORB_create(max_features)

    # compute keypoints and features
    (keypoints, features) = orb.detectAndCompute(gray_image, None)

    if show_details:
        show_keypoints(image, keypoints)

    return keypoints, features

In [14]:
def match_features(features_template, features_image) -> List[List[cv2.DMatch]]:
    """
    Match features from the source image with the features from the destination image.
    :return: [[DMatch]] - The rusult of the matching. For each set of features from the source image,
    it returns the first 'K' matchings from the destination images.
    """

    method = cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING
    matcher = cv2.DescriptorMatcher_create(method)
    matches = matcher.match(features_template, features_image, None)

    return matches

In [15]:
def generate_homography(all_matches:  List[cv2.DMatch],
                        descsA: List[cv2.KeyPoint],
                        descsB : List[cv2.KeyPoint],
                        ratio: float = 0.75,
                        ransac_rep: int = 4.0):
    """
    :param all_matches [DMatch]
    :param descsA [cv.Point]
    :param ratio - Lowe's ratio test (the ratio 1st neighbour distance / 2nd neighbour distance)
    :param descsA: nd.array [Nx2] (x, y coordinates)
    :param descsB: nd.array [Nx2] (x, y coordinates)
    :param ransac_rep: float. The threshold in the RANSAC algorithm.
    :return: The homography matrix.
    
    class DMatch:
        distance - Distance between descriptors. The lower, the better it is.
        imgIdx - Index of the train image
        queryIdx - Index of the descriptor in query descriptors
        trainIdx - Index of the descriptor in train descriptors
    
    class KeyPoint:
        pt - The x, y coordinates of a point.
    
    """
    if not all_matches:
        return None
    # allocate memory for the keypoints (x, y)-coordinates from the
	# top matches -- we'll use these coordinates to compute our
	# homography matrix
    ptsA = np.zeros((len(all_matches), 2), dtype="float")
    ptsB = np.zeros((len(all_matches), 2), dtype="float")
	# loop over the top matches

    for (i, m) in enumerate(all_matches):
		# indicate that the two keypoints in the respective images
		# map to each other
        ptsA[i] = descsA[m.trainIdx].pt
        ptsB[i] = descsB[m.queryIdx].pt
		
    # compute the homography matrix between the two sets of matched points
    (H, mask) = cv2.findHomography(ptsA, ptsB, method=cv2.RANSAC)

    return H

In [16]:
def align_images(template, image, maxFeatures=1000, keepPercent=0.2, debug=False):

	(kpsA, descsA) = get_keypoints_and_features(template, max_features=maxFeatures)
	(kpsB, descsB) = get_keypoints_and_features(image, max_features=maxFeatures)

	if debug:
		print(f"Keypoints lenght: {len(kpsA)},/n Features lenght: {len(descsA)}")
		print(f"Keypoints lenght: {len(kpsB)},/n Features lenght: {len(descsB)}")

	# match the features
	matches = match_features(descsA, descsB)

	if debug:
		print(f"Total number of matches: {len(matches)}")

    # sort the matches by their distance (the smaller the distance,
	# the "more similar" the features are)
	matches = sorted(matches, key=lambda x:x.distance)
	# keep only the top matches
	keep = int(len(matches) * keepPercent)
	matches = matches[:keep]
	# check to see if we should visualize the matched keypoints
	if debug:
		matchedVis = cv2.drawMatches(template, kpsA, image, kpsB, matches, None)
		matchedVis = imutils.resize(matchedVis, width=1000)
		cv2.imshow("Matched Keypoints", matchedVis)
		cv2.waitKey(0)
		
	# compute homography matrix 
	H = generate_homography(matches,kpsB,kpsA)

	# use the homography matrix to align the images
	(h, w) = template.shape[:2]
	aligned = cv2.warpPerspective(image, H, (w, h))
	# return the aligned image
	if debug:
		show_image(aligned,window_name="Alligned image")

	return aligned
	

In [17]:
class Line:
    """
    Store a line based on the two points.
    """
    def __init__(self, point_1, point_2):
        self.point_1 = point_1
        self.point_2 = point_2
        

class Point:
    def __init__(self, x, y):
        self.x = np.int32(np.round(x))
        self.y = np.int32(np.round(y))
    
    def get_point_as_tuple(self):
        return (self.x, self.y)
    
        
class Patch:
    """
    Store information about each item (where the mark should be found) in the table. 
    """
    def __init__(self, image_patch, x_min, y_min, x_max, y_max, line_idx, column_idx):
        self.image_patch = image_patch
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max
        self.line_idx = line_idx
        self.column_idx = column_idx
        self.has_domino: int = 0 # 0 meaning it does not contain an 'X', 1 meaning it contains an 'X'
    
    def set_domino(self, has_domino: int):
        assert has_domino == False or has_domino == True # convention 
        self.has_domino = has_domino

In [18]:
def draw_lines(image, lines: [Line], timeout: int = 0, color: tuple = (0, 0, 255),
               return_drawing: bool = False, window_name: str = 'window'):
    """
    Plots the lines into the image.
    :param image.
    :param lines.
    :param timeout. How many seconds to wait untill it close the window.
    :param color. The color used to draw the lines
    :param return_drawing. Use it if you want the drawing to be return instead of displayed.
    :return None if return_drawing is False, otherwise returns the drawing.
    """
    drawing = image.copy()
    if drawing.ndim == 2:
        drawing = cv2.cvtColor(drawing, cv2.COLOR_GRAY2BGR)
    for line in lines: 
        cv2.line(drawing, line.point_1.get_point_as_tuple(), line.point_2.get_point_as_tuple(), color, 2, cv2.LINE_AA)
        
    if return_drawing:
        return drawing
    else:
        show_image(drawing, window_name=window_name, timeout=timeout)

In [19]:
def show_patches_individual(patches: [Patch], show_each_patch: False) -> None:
    """
    This function draws a colored rectangle if the patch has an 'X'. 
    """
 
    image_color = np.zeros((2000, 2000, 3), np.uint8) # it something crashed it is because of this dimension, it may be too small.
    x_min = patches[0].x_min
    y_min = patches[0].y_min
    for patch in patches:
        x_min_current = patch.x_min - x_min
        y_min_current = patch.y_min - y_min
        x_max_current = patch.x_max - x_min
        y_max_current = patch.y_max - y_min
        image_color[y_min_current: y_max_current, x_min_current: x_max_current] = np.dstack((patch.image_patch, patch.image_patch, patch.image_patch))

        if patch.has_x == 1:  
            cv2.rectangle(image_color, (x_min_current, y_min_current), 
                         (x_max_current, y_max_current), color=(255, 0, 0), thickness=1)
        if show_each_patch:
            show_image(patch, window_name='one patch', timeout=0)
         
    show_image(image_color, window_name='patches', timeout=0)

In [20]:
def get_patches(lines: [Line], columns: [Line], image, show_patches: bool = False) -> [Patch]:
    """
    It cuts out each box from the table defined by the lines and columns.
    :param lines. The lines that difine the table.
    :param columns. The columns that difine the table.
    :param image. The image containing the table.
    :param show_patches. Determine if the patches will be drawn on the image or not.
    :return : A list with all boxes in the table.
    """
    ################ piece_placed_wrong_error check tommorow how to take a bigger patch 
    def crop_patch(image_, x_min, y_min, x_max, y_max , piece_placed_wrong_error = 5):
        """
        Crops the bounding box represented by the coordinates.
        """
        return image_[y_min-10: y_max+10, x_min-10: x_max+10].copy()
    
    def draw_patch(image_, patch: Patch, color: tuple = (255, 0, 255)):
        """
        Draw the bounding box corresponding to the patch on the image.
        """
        cv2.rectangle(image_, (patch.x_min, patch.y_min), (patch.x_max, patch.y_max), color=color, thickness=5)
    
    assert image.ndim == 2
    if show_patches: 
        image_color = np.dstack((image, image, image))
  
    lines.sort(key=lambda line: line.point_1.y)
    columns.sort(key=lambda column: column.point_1.x)
    patches = []
    step = 5
    for line_idx in range(len(lines) - 1):
        for col_idx in range(len(columns) - 1):
            current_line = lines[line_idx]
            next_line = lines[line_idx + 1] 
            
            y_min = current_line.point_1.y + step
            y_max = next_line.point_1.y - step
            
            current_col = columns[col_idx]
            next_col = columns[col_idx + 1]
            x_min = current_col.point_1.x + step 
            x_max = next_col.point_1.x - step
            
            patch = Patch(image_patch=crop_patch(image,  x_min, y_min, x_max, y_max),
                          x_min=x_min, y_min=y_min, x_max=x_max, y_max=y_max,
                          line_idx=line_idx, column_idx=col_idx)

            # insert here the moethod that adds tha has domino 

            if show_patches:
                draw_patch(image_color, patch)
            
            patches.append(patch)
            
    if show_patches:
        show_image(image_color, window_name='Patches', timeout=0)

    return patches

In [21]:
def find_patches_template(image, template, show_intermediate_results: bool = False):
    """
    This function finds the 'X' in the entire image using the following steps: 
    1. transforming the query image and the template to grayscale
    2. getting the edges for both images using Canny edge detector with threshold1=100 and threshold2=150
    3. obtaining the displacement vector between the query image and the template image
    4. allign the query image to the template
    5. obtain the first left/right corner of the first table and the second table
    6. getting the vertical/horizontal lines based on the table dimension and the number of vertical/horizontal lines based
    7. cut out the paches (the boxes) from the two tables based on the horizontal and vertical lines 
    """

    alligned_image = align_images(template, image, debug=show_intermediate_results)

    # top_left = Point(x=9, y=8)
    # bottom_right = Point(x=1905, y=1921)

    top_left = Point(x=10, y=11)
    bottom_right = Point(x=1902, y=1922)

    step_x =(bottom_right.x-top_left.x)/15
    step_y =(bottom_right.y-top_left.y)/15
     
    
    def get_vertical_and_horizontal_lines(left_corner: Point, step_x_, step_y_, window_size_): 
        vertical_lines = []
        horizontal_lines = []
        
        # TODO: compute the vertical/horizontal lines based left corner step and window_size 
        # ! DO i still need window_size tho? debatable
        for i in range(0,16):  
            # x_min=np.uint64(left_corner.x+i*step_x_+window_size_)
            x_min=np.uint64(left_corner.x+i*step_x_)
            line = Line(Point(x=x_min, y=0), Point(x=x_min, y=5000))
            vertical_lines.append(line)
        for i in range(0,16):  
            # y_min=np.uint64(left_corner.y+i*step_y_+window_size_)
            y_min=np.uint64(left_corner.y+i*step_y_)
            line = Line(Point(x=0, y=y_min), Point(x=5000, y=y_min))
            horizontal_lines.append(line)   
        
        return vertical_lines, horizontal_lines
     
    
    # window_size = np.abs([pos_i, pos_j]).max()
    vertical_lines_left, horizontal_lines_left = get_vertical_and_horizontal_lines(top_left, step_x, step_y, 
                                                                                   window_size_= 10) 
    
    alligned_image_gray = cv2.cvtColor(alligned_image,cv2.COLOR_BGR2GRAY)
    patches = get_patches(horizontal_lines_left, vertical_lines_left, alligned_image_gray, 
                          show_patches=show_intermediate_results)
 
    
    return patches

In [12]:
def show_each_patch(all_patches):
    for patch in all_patches:
        cimg = cv2.cvtColor(patch.image_patch,cv2.COLOR_GRAY2BGR)
        show_image(cimg ,resize=False)

In [22]:
class IsDominoClassifier:
    """
    A strong classifier that detects if the patch has a domino piece or not.
    The classifier will classify wrong the patches from the center of the board
    but that problem is dealt with later.
    """
    def __init__(self):
        self.white_lower_bound = 160
        self.black_upper_bound = 50
        self.percent_threshold = 72

    def check_black_white_concentration(self,
                                        image,
                                        line_idx,
                                        column_idx,
                                        show_hist = False,
                                        show_img = False):

        if show_hist:
            hist = cv2.calcHist([image],[0],None,[256],[0,256])
            plt.plot(hist)
            plt.show()
        
        if show_img:
            show_image(image, resize=False)

        # sa adaug use case specific daca pach ul e in centru
        pixels_number = image.shape[0] * image.shape[1]
        black_ish_pixel_nb = np.count_nonzero((image<self.black_upper_bound))
        white_ish_pixel_nb = np.count_nonzero((image>self.white_lower_bound))
        percent = ((white_ish_pixel_nb + black_ish_pixel_nb) / pixels_number) * 100

        return percent

    def check_white_concentration(self,
                                  image,
                                  line_idx,
                                  column_idx,
                                  show_hist = False,
                                  show_img = False):

        if show_hist:
            hist = cv2.calcHist([image],[0],None,[256],[0,256])
            plt.plot(hist)
            plt.show()
        
        if show_img:
            show_image(image, resize=False)

        # sa adaug use case specific daca pach ul e in centru
        pixel_nb = image.shape[0] * image.shape[1]
        white_ish_pixel_nb = np.count_nonzero((image>180))
        percent = ((white_ish_pixel_nb) / pixel_nb) * 100

        return percent

    def has_domino(self,
                patch: Patch,
                sh:bool= False,
                si:bool=False):
        """
        Receive a Patch and return True if there is a domino in the patch else False.
        """ 
        percent = self.check_black_white_concentration(patch.image_patch,
                                                       patch.line_idx,
                                                       patch.column_idx,
                                                       sh,
                                                       si)
        if percent > self.percent_threshold:
            return True, percent
        
        return False, percent


In [23]:
class DominoClassifier:
    """
    A classifier that classifies the domino in one of the following classes [0,1,2,3,4,5,6]
    """
    def __init__(self):
        pass   

    def classify(self, patch: Patch, show_image: bool = True) -> int:
        """
        Receive a Patch and return patch class.
        """ 
        img = cv2.GaussianBlur(patch.image_patch, (7, 7), 50)
        cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
        circles = cv2.HoughCircles(img,
                                   cv2.HOUGH_GRADIENT_ALT,
                                   1,
                                   20,
                                   param1=200,
                                   param2=0.96,
                                   minRadius=-1,
                                   maxRadius=-1)
        if circles is None:
            if show_image:    
                cv2.imshow('detected circles',cimg)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
            return 0
        else:
            if show_image:
                circles = np.uint16(np.around(circles))
                for i in circles[0,:]:
                # draw the outer circle
                    cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
                    # draw the center of the circle
                    cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)
                cv2.imshow('detected circles',cimg)
                cv2.waitKey(0)
                cv2.destroyAllWindows()

            return len(circles[0,:])


In [24]:
def classify_patches_with_classifiers(patches: [Patch], show_patches:bool = False) -> None:
    """
    Receive the patches and classify if the patch contains a domino or not.
    Also classify what type of domino is.
    :param patches.
    :return None
    """
    is_piece = IsDominoClassifier()
    piece_cls = DominoClassifier()

    def get_white_concentration(image,
                                show_hist = False,
                                show_img = False):

        if show_hist:
            hist = cv2.calcHist([image],[0],None,[256],[0,256])
            plt.plot(hist)
            plt.show()
        
        if show_img:
            show_image(image, resize=False)

        pixel_nb = image.shape[0] * image.shape[1]
        white_ish_pixel_nb = np.count_nonzero((image>180))
        percent = ((white_ish_pixel_nb) / pixel_nb) * 100

        return percent
    
    def in_middle_box(line_idx: int, column_idx: int) -> bool:
        middle_box_line_min = 6
        middle_box_line_max = 7
        middle_box_column_min = 6
        middle_box_column_max = 8

        if line_idx >= middle_box_line_min and line_idx <= middle_box_line_max:
            if column_idx >= middle_box_column_min and column_idx <= middle_box_column_max:
                return True
        
        return False
##### BREEEAAK HERE
    # get all patches that are supposed to be dominoes 
    bigger_than_80 = []

    for i in range(0,225):
        result = is_piece.has_domino(patches[i])
        if result[0]:
            bigger_than_80.append((i,result[1],None))

    # if show_patches:
    #     for image in bigger_than_80:
    #         show_image(patches[image[0]].image_patch)

    all_dominoes_on_board = []

    for tpl in bigger_than_80:
        # get a supposed class for each patch 
        supposed_class = piece_cls.classify(patches[tpl[0]],show_image=show_patches)
        # print("Image line {} column {}".format(patches[tpl[0]].line_idx,patches[tpl[0]].column_idx))
        # print("Number of circles {}".format(supposed_class))
        if not in_middle_box(patches[tpl[0]].line_idx,patches[tpl[0]].column_idx):
            all_dominoes_on_board.append([patches[tpl[0]],supposed_class])
        else:
            if supposed_class > 0:
                all_dominoes_on_board.append([patches[tpl[0]],supposed_class])
            else:
                white_concentration = get_white_concentration(patches[tpl[0]].image_patch)
                # the logic is that the only case where the patch is not verry white and has 0 as class
                # is when the is when the patch does not have a domino but was extracted as one because
                # the midlle patches are simmilar to dominoes
                if white_concentration > 80:
                    all_dominoes_on_board.append([patches[tpl[0]],supposed_class])

    return all_dominoes_on_board

In [25]:
class DDD_Game():
    def __init__(self) -> None:
        self.game_matrix=[[[False, 0, None, -1] for j in range(15)] for i in range(15)]
        points= [[5, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 5],
                [0, 0, 3, 0, 0, 4, 0, 0, 0, 4, 0, 0, 3, 0, 0],
                [0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0],
                [4, 0, 0, 3, 0, 2, 0, 0, 0, 2, 0, 3, 0, 0, 4],
                [0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0],
                [0, 4, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 4, 0],
                [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
                [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 4, 0, 2, 0, 1, 0, 0, 0, 1, 0, 2, 0, 4, 0],
                [0, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0],
                [4, 0, 0, 3, 0, 2, 0, 0, 0, 2, 0, 3, 0, 0, 4],
                [0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0],
                [0, 0, 3, 0, 0, 4, 0, 0, 0, 4, 0, 0, 3, 0, 0],
                [5, 0, 0, 4, 0, 0, 0, 3, 0, 0, 0, 4, 0, 0, 5]]
        for line in range(0,15):
            for column in range(0,15):
                self.game_matrix[line][column][1] = points[line][column]
        self.col_id_to_letter_mapping = {0: 'A',
                                          1: 'B',
                                          2: 'C',
                                          3: 'D',
                                          4: 'E',
                                          5: 'F',
                                          6: 'G',
                                          7: 'H',
                                          8: 'I',
                                          9: 'J',
                                          10: 'K',
                                          11: 'L',
                                          12: 'M', 
                                          13: 'N', 
                                          14: 'O'}
        self.player_pos_to_piece = [-1, 1, 2, 3, 4, 5, 6, 0, 2, 5, 3, 4, 6, 2, 2, 0, 3, 5, 4, 1, 6, 2, 4, 5, 5, 0, 6, 3, 4, 2, 0, 1, 5, 1, 3, 4, 4, 4, 5, 0, 6, 3, 5, 4, 1, 3, 2, 0, 0, 1, 1, 2, 3, 6, 3, 5, 2, 1, 0, 6, 6, 5, 2, 1, 2, 5, 0, 3, 3, 5, 0, 6, 1, 4, 0, 6, 3, 5, 1, 4, 2, 6, 2, 3, 1, 6, 5, 6, 2, 0, 4, 0, 1, 6, 4, 4, 1, 6, 6, 3, 0]
        self.player_red_points = 0
        self.player_green_points = 0
    # added -1 here and also change smt ctrz z if acc lower
    def add_new_dominos(self,all_dominos, turn_id):

        print(50*"*")
        points = 0
        new_pieces_class = []
        write_in_file = []
        for domino_and_class in all_dominos:

            line = domino_and_class[0].line_idx
            col = domino_and_class[0].column_idx
            if self.game_matrix[line][col][0] == False:
                self.game_matrix[line][col][0] = True
                self.game_matrix[line][col][2] = domino_and_class[0]
                self.game_matrix[line][col][3] = domino_and_class[1]
                new_pieces_class.append(domino_and_class[1])

                if self.game_matrix[line][col][1] != 0 :
                    points += self.game_matrix[line][col][1]
                    
                write_in_file.append(str(line+1)+self.col_id_to_letter_mapping[col]+" "+str(domino_and_class[1]))
                print(str(line+1)+self.col_id_to_letter_mapping[col]+" "+str(domino_and_class[1]))

        if len(set(new_pieces_class)) == 1:
            points = points * 2
        

        add_to_red = 0
        add_to_green = 0

        # check who get s 3 poits also
        # if a piece equal to any of their position they get 3 points 
        if self.player_green_points != 0:
            if self.player_pos_to_piece[self.player_green_points] in set(new_pieces_class):
                add_to_green = 3

        if self.player_red_points != 0:
            if self.player_pos_to_piece[self.player_red_points] in set(new_pieces_class):
                add_to_red = 3

        if turn_id % 2 == 0:
            self.player_green_points+=(add_to_green + points)
            self.player_red_points += add_to_red
            print(add_to_green + points)
            write_in_file.append(str(add_to_green + points))
        else:
            self.player_red_points+=(add_to_red + points)
            self.player_green_points +=add_to_green
            print(add_to_red + points)
            write_in_file.append(str(add_to_red + points))
            
        return write_in_file
        

In [26]:
def proccess_game(images_path,save_out,template_path, nr_game_to_be_processed:int):
    for game_id in range(1,nr_game_to_be_processed+1):
        game = DDD_Game()
        image_ids = [str(i) if i/10>=1.0 else ("0"+str(i)) for i in range(1,21)]
        player_turn = 0
        for image_id in image_ids:

            image = cv2.imread(os.path.join(images_path, str(game_id) + "_" + image_id + ".jpg"))
            template = cv2.imread(template_path)

            patches = find_patches_template(image,
                                            template,
                                            show_intermediate_results=False)

            all_dominos = classify_patches_with_classifiers(patches,show_patches=False)
            event_list = game.add_new_dominos(all_dominos,player_turn)

            f = open(os.path.join(save_out, str(game_id) + "_" + image_id + ".txt"), 'w')

            for line in event_list[0:-1]:
                f.write(str(line)+'\n')
            
            f.write(event_list[-1])
            f.close()
            player_turn+=1

In [27]:
class Rules:
    def __init__(self) -> None:
        pass

    @staticmethod
    def check_box_full(box):

        nr_doms = 0 

        for line in box:
            for el in line:
                if el[0] == True:
                    nr_doms+=1
        
        if nr_doms == 4:
            return True
        
        return False

In [28]:
def resolve_bonus(images_path, textes_out, template_path):
    image_ids = [str(i) if i/10>=1.0 else ("0"+str(i)) for i in range(1,11)]
    # image_ids = ["02"]
    for image_id in image_ids:
        game = DDD_Game()
        rules = Rules()
        wrong_nr = 0
        write_in_file = []
        image = cv2.imread(os.path.join(images_path,image_id + ".jpg"))
        template = cv2.imread(template_path)
        patches = find_patches_template(image,
                                        template,
                                        show_intermediate_results=False)   
        all_dominos = classify_patches_with_classifiers(patches,
                                                        show_patches=False) 
        game.add_new_dominos(all_dominos,0)
        # iterate throught the matrix with a sliding window of 2 by 2 and check 
        # if any of the rules are broken 
        print(game.game_matrix)
        for i in range(0, 14, 1):
            for j in range(0, 14, 1):
                window = [[game.game_matrix[i][j], game.game_matrix[i][j+1]],
                        [game.game_matrix[i+1][j], game.game_matrix[i+1][j+1]]]
                
                # check if box has 4 dominoes
                if Rules.check_box_full(window):
                    wrong_nr += 1
                    for line in window:
                        for el in line:
                            write_in_file.append(str(el[2].line_idx+1) + game.col_id_to_letter_mapping[el[2].column_idx])
                    
        f = open(os.path.join(textes_out, image_id + ".txt"), 'w')
        f.write(str(wrong_nr)+'\n')

        for line in write_in_file:
            f.write(str(line)+'\n')
            
        f.close()


In [29]:
proccess_game("Z:/Master II/CV - Computer Vision/zDouble-Double-Dominoes/test/regular_tasks",
              "Z:/Master II/CV - Computer Vision/zDouble-Double-Dominoes/test/predictions/regular_tasks",
              "Z:/Master II/CV - Computer Vision/zDouble-Double-Dominoes/Rares_Patrascu_CV/template/best_templ.jpg",
              5)

**************************************************
8H 6
9H 3
0
**************************************************
6H 5
7H 6
0
**************************************************
5G 4
6G 5
1
**************************************************
6I 5
6J 6
1
**************************************************
4F 1
4G 4
5
**************************************************
2F 2
3F 1
4
**************************************************
4D 0
4E 1
3
**************************************************
5D 0
6D 2
5
**************************************************
6K 6
6L 6
4
**************************************************
6M 6
6N 2
7
**************************************************
5B 3
5C 0
5
**************************************************
6B 3
7B 6
7
**************************************************
8A 1
8B 6
3
**************************************************
9B 6
10B 6
8
**************************************************
11B 6
11C 0
2
**************************************************
1

In [30]:
resolve_bonus("Z:/Master II/CV - Computer Vision/zDouble-Double-Dominoes/test/bonus_task",
              "Z:/Master II/CV - Computer Vision/zDouble-Double-Dominoes/test/predictions/bonus_task",
              "Z:/Master II/CV - Computer Vision/zDouble-Double-Dominoes/Rares_Patrascu_CV/template/best_templ.jpg") 

**************************************************
5I 5
6G 3
6H 2
6I 2
7E 4
7H 2
7I 2
7J 6
8E 1
8F 1
8G 4
8H 4
8J 6
9E 1
9J 5
9K 5
9L 1
9M 1
10E 0
10M 6
10N 5
10O 6
11E 0
12A 3
12B 5
12E 0
13B 1
13E 0
14B 1
14C 1
14D 2
14E 2
21
[[[False, 5, None, -1], [False, 0, None, -1], [False, 0, None, -1], [False, 4, None, -1], [False, 0, None, -1], [False, 0, None, -1], [False, 0, None, -1], [False, 3, None, -1], [False, 0, None, -1], [False, 0, None, -1], [False, 0, None, -1], [False, 4, None, -1], [False, 0, None, -1], [False, 0, None, -1], [False, 5, None, -1]], [[False, 0, None, -1], [False, 0, None, -1], [False, 3, None, -1], [False, 0, None, -1], [False, 0, None, -1], [False, 4, None, -1], [False, 0, None, -1], [False, 0, None, -1], [False, 0, None, -1], [False, 4, None, -1], [False, 0, None, -1], [False, 0, None, -1], [False, 3, None, -1], [False, 0, None, -1], [False, 0, None, -1]], [[False, 0, None, -1], [False, 3, None, -1], [False, 0, None, -1], [False, 0, None, -1], [False, 2, None, -